In [5]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D
import numba as nb
import math

In [6]:
import json
with open("../HggAnalysisDev/Preselection/metadata/chunklist.json") as f:
    chunks = json.load(f) 

In [7]:
#opening signal samples (HHggtautau)
# file = uproot.open(chunks["signal"][0][0])
# opening HHggbb
file = uproot.open('/hadoop/cms/store/user/legianni/skimNano-Hggselection-wFjets/GluGluToHHTo2B2G_node_SM_13TeV-madgraph_private_mc17/skimNano-Hggselection__v5/210304_162411/0000/tree_1.root')
tree = file['Events']
#Retrieving genPart keys
gen_keys = tree.keys(filter_name="GenPart_*")
genJet_keys = tree.keys(filter_name="GenJet_*")

In [8]:
genPart = tree.arrays(gen_keys + ["genWeight"], library="ak", how="zip") 
#selecting photons from Higgs and Higgs bosons decaying to photons
mask_genPho = genPart.GenPart.pdgId == 22      #all photons
mIdx = genPart.GenPart.genPartIdxMother        #motherIdx
mask_fromH = genPart.GenPart.pdgId[mIdx] == 25 #all mothers H

genPho_fromH = genPart.GenPart[mask_genPho & mask_fromH] #photons from H

HggIdx = genPho_fromH.genPartIdxMother[:,0] #motherIdx of gg_fromH (taking the first photon)


In [9]:
@nb.jit
def getHPt(genPart, HggIdx):
    hPt = np.zeros(len(genPart))
    for i in range(len(hPt)):
        hIndex = int(HggIdx[i])
        hPt[i] = genPart.GenPart.pt[i][hIndex]
    return hPt

In [10]:
genHgg_pt = getHPt(genPart,HggIdx)

In [11]:
@nb.jit
def deltaphi_devfunc(phi1, phi2):
    dphi = phi1 - phi2
    out_dphi = float(0)
    
    if dphi > math.pi:
        dphi = dphi - 2 * math.pi
        out_dphi = dphi
    elif (dphi + math.pi) < 0:
        dphi = dphi + 2 * math.pi
        out_dphi = dphi
    else:
        out_dphi = dphi
    return out_dphi

@nb.jit
def get_deltaR(taus):
    nEvents = len(taus)
    dRs = np.empty(nEvents, np.float32)
    for i in range(nEvents):
        tau1 = taus[i][0]
        tau2 = taus[i][1]
        dR = np.sqrt((tau1.eta-tau2.eta)**2 + deltaphi_devfunc(tau1.phi, tau2.phi)**2)  
        dRs[i] = dR
    return dRs

dRs_dipho = get_deltaR(genPho_fromH)

In [12]:
Hist2D((dRs_dipho , genHgg_pt),bins=[np.linspace(0,4,101),np.linspace(0,1000,151)])

[[ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    1.00 ± 1.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 ...
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]]